# Banknote Authentication – 1-Qubit QML vs Classical Models (ES, λ=24, 60 gens)

Esta libreta implementa el experimento de autenticación de billetes con:

- Un clasificador cuántico de **1 qubit** entrenado con *Evolution Strategies* (ES).
- Modelos clásicos de referencia: Regresión Logística y MLP.

Hiperparámetros principales del QML:

- $\mu = 5$ padres  
- $\lambda = 24$ hijos por generación  
- $\alpha = 0.1$ (escala del ruido)  
- 60 generaciones  
- Hasta 600 muestras de entrenamiento para el QML  
- 256 *shots* por evaluación

In [ ]:
# === 1. Imports and basic config ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# SpinQit / SpinQ
try:
    from spinqkit import get_basic_simulator, get_compiler, Circuit, Rx, Ry, Rz, BasicSimulatorConfig
except ImportError:
    try:
        from spinqkit import get_basic_simulator, get_compiler, Circuit, Rx, Ry, Rz
        from spinqkit.backends import BasicSimulatorConfig
    except ImportError as e:
        raise ImportError(
            "No pude importar BasicSimulatorConfig desde spinqkit. "
            "Verifica la instalación / versión de SpinQit."
        ) from e

np.random.seed(42)
print("Imports OK.")

In [ ]:
# === 2. Load and preprocess Banknote dataset ===

def load_data():
    """Carga el dataset Banknote desde archivo local o desde UCI."""
    import numpy as _np
    try:
        data = _np.loadtxt("data_banknote_authentication.txt",
                           delimiter=",", dtype=_np.float64)
    except Exception:
        import urllib.request
        url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
        print("Descargando datos desde UCI...")
        urllib.request.urlretrieve(url, "data_banknote_authentication.txt")
        data = _np.loadtxt("data_banknote_authentication.txt",
                           delimiter=",", dtype=_np.float64)
    return data

data = load_data()
x_raw = data[:, :-1]
y = data[:, -1].astype(int)

print("Raw shape:", x_raw.shape, "labels shape:", y.shape)

# Escalado columna a columna a [0, pi]
x_scaled = x_raw.copy()
for col in range(x_scaled.shape[1]):
    mini, maxi = np.min(x_scaled[:, col]), np.max(x_scaled[:, col])
    x_scaled[:, col] = np.pi * (x_scaled[:, col] - mini) / (maxi - mini)

# Features 2D: x1 = sqrt(var+skew), x2 = sqrt(curtosis)
x2d = np.empty((x_scaled.shape[0], 2), dtype=np.float32)
x2d[:, 0] = np.sqrt(x_scaled[:, 0] + x_scaled[:, 1])
x2d[:, 1] = np.sqrt(x_scaled[:, 2])

print("2D features shape:", x2d.shape)

X_train, X_test, y_train, y_test = train_test_split(
    x2d, y, test_size=0.2, stratify=y, random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Test: ", X_test.shape, y_test.shape)

In [ ]:
# === 2b. Scatter plot of feature space (Fig. 8 style) ===

plt.figure(figsize=(5, 5))
mask0 = (y == 0)
mask1 = (y == 1)
plt.scatter(x2d[mask0, 0], x2d[mask0, 1], c='red', s=20, label='Class 0')
plt.scatter(x2d[mask1, 0], x2d[mask1, 1], c='blue', s=20, label='Class 1')
plt.xlabel(r"$\sqrt{\mathrm{variance} + \mathrm{skewness}}$")
plt.ylabel("Curtosis")
plt.title("Decision space of the BNA dataset\n(after feature extraction and selection)")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
# === 3. One-qubit Quantum Classifier with ES (lambda=24, 60 gens) ===

class QuantumClassifier:
    """Modelo QML de 1 qubit con 3 parámetros."""
    def __init__(self, n_params=3, shots=256):
        self.n_params = n_params
        self.shots = shots
        self.params = np.zeros(n_params, dtype=np.float64)
        self.engine = get_basic_simulator()
        self.compiler = get_compiler("native")

    def circuit(self, x, params):
        x = np.asarray(x, dtype=np.float64).flatten()
        params = np.asarray(params, dtype=np.float64).flatten()
        if x.size != 2:
            raise ValueError(f"x must be 2D, got {x.shape}")
        if params.size != 3:
            raise ValueError(f"params must have length 3, got {params.shape}")

        circ = Circuit()
        q = circ.allocateQubits(1)

        # Codificación de datos
        circ << (Ry, q[0], float(x[1]))
        circ << (Rz, q[0], float(x[0]))

        # Capa variacional
        circ << (Rx, q[0], float(params[0]))
        circ << (Rz, q[0], float(params[1]))
        circ << (Ry, q[0], float(params[2]))

        return circ

    def _run_and_prob1(self, x, params):
        circ = self.circuit(x, params)
        exe = self.compiler.compile(circ, 0)

        config = BasicSimulatorConfig()
        config.configure_shots(self.shots)

        result = self.engine.execute(exe, config)
        counts = result.counts
        total = sum(counts.values())
        if total == 0:
            return 0.5
        p1 = counts.get('1', 0) / total
        p1 = float(np.clip(p1, 1e-7, 1 - 1e-7))
        return p1

    def predict_proba(self, X, params=None):
        if params is None:
            params = self.params
        X = np.atleast_2d(X)
        probs = np.zeros(X.shape[0], dtype=np.float64)
        for i, x in enumerate(X):
            probs[i] = self._run_and_prob1(x, params)
        return probs

    def predict(self, X, params=None, threshold=0.5):
        p = self.predict_proba(X, params)
        return (p > threshold).astype(int)

    def accuracy(self, params, X, y):
        y = np.asarray(y, dtype=int)
        y_pred = self.predict(X, params=params)
        return accuracy_score(y, y_pred)

    def fit_es_fast(self, X, y, mu=5, lam=24, alpha=0.1, iters=60,
                    max_train_samples=600, seed=None, verbose=True):
        """Entrenamiento ES.

        - mu=5 padres, lam=24 hijos
        - iters=60 generaciones
        - max_train_samples: máximo de muestras para acelerar el QML
        """
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y, dtype=int)

        rng = np.random.default_rng(seed)

        # Submuestreo para acelerar
        if max_train_samples is not None and X.shape[0] > max_train_samples:
            idx = rng.choice(X.shape[0], size=max_train_samples, replace=False)
            X = X[idx]
            y = y[idx]

        # Inicializar padres
        parents = rng.uniform(-np.pi/4, np.pi/4, size=(mu, self.n_params))
        acc_parents = np.array([self.accuracy(p, X, y) for p in parents])
        best_idx = int(np.argmax(acc_parents))
        best_params = parents[best_idx].copy()
        best_acc = float(acc_parents[best_idx])

        if verbose:
            print(f"[ES] Init best accuracy = {best_acc:.4f}")

        for gen in range(iters):
            children = np.zeros((lam, self.n_params), dtype=np.float64)
            for j in range(lam):
                parent_idx = rng.integers(mu)
                noise = alpha * rng.normal(0.0, 1.0, size=self.n_params)
                children[j] = parents[parent_idx] + noise

            acc_children = np.array([self.accuracy(ch, X, y) for ch in children])
            idx_sorted = np.argsort(-acc_children)
            parents = children[idx_sorted[:mu]]
            acc_parents = acc_children[idx_sorted[:mu]]

            if acc_parents[0] > best_acc:
                best_acc = float(acc_parents[0])
                best_params = parents[0].copy()

            if verbose and (gen % 10 == 0 or gen == iters-1):
                print(f"[ES] Gen {gen+1:2d}/{iters}: best_acc={best_acc:.4f}")

        self.params = best_params
        if verbose:
            print(f"[ES] Finished. Best accuracy = {best_acc:.4f}")
        return best_params

In [ ]:
# === 4. Single-run experiment ===

def run_single_experiment(random_state=0, verbose=True):
    X_tr, X_ts, y_tr, y_ts = train_test_split(
        x2d, y, test_size=0.2, stratify=y, random_state=random_state
    )

    if verbose:
        print(f"[Run {random_state}] Train {X_tr.shape}, Test {X_ts.shape}")

    # QML con ES (mu=5, lam=24, iters=60)
    qml = QuantumClassifier(n_params=3, shots=256)
    qml.fit_es_fast(X_tr, y_tr,
                    mu=5, lam=24, alpha=0.1, iters=60,
                    max_train_samples=600,
                    seed=random_state, verbose=False)
    y_tr_q = qml.predict(X_tr)
    y_ts_q = qml.predict(X_ts)
    acc_tr_q = accuracy_score(y_tr, y_tr_q)
    acc_ts_q = accuracy_score(y_ts, y_ts_q)

    # Regresión logística
    lr = LogisticRegression(max_iter=500, random_state=random_state)
    lr.fit(X_tr, y_tr)
    y_tr_lr = lr.predict(X_tr)
    y_ts_lr = lr.predict(X_ts)
    acc_tr_lr = accuracy_score(y_tr, y_tr_lr)
    acc_ts_lr = accuracy_score(y_ts, y_ts_lr)

    # MLP (clásico)
    mlp = MLPClassifier(hidden_layer_sizes=(50, 50),
                        activation='relu',
                        solver='adam',
                        learning_rate_init=0.001,
                        max_iter=300,
                        random_state=random_state)
    mlp.fit(X_tr, y_tr)
    y_tr_mlp = mlp.predict(X_tr)
    y_ts_mlp = mlp.predict(X_ts)
    acc_tr_mlp = accuracy_score(y_tr, y_tr_mlp)
    acc_ts_mlp = accuracy_score(y_ts, y_ts_mlp)

    if verbose:
        print(f"  QML train/test: {acc_tr_q:.4f} / {acc_ts_q:.4f}")
        print(f"  LR  train/test: {acc_tr_lr:.4f} / {acc_ts_lr:.4f}")
        print(f"  MLP train/test: {acc_tr_mlp:.4f} / {acc_ts_mlp:.4f}")

    return {
        "X_tr": X_tr, "y_tr": y_tr,
        "X_ts": X_ts, "y_ts": y_ts,
        "qml": qml, "lr": lr, "mlp": mlp,
        "acc_tr_q": acc_tr_q, "acc_ts_q": acc_ts_q,
        "acc_tr_lr": acc_tr_lr, "acc_ts_lr": acc_ts_lr,
        "acc_tr_mlp": acc_tr_mlp, "acc_ts_mlp": acc_ts_mlp,
        "seed": random_state
    }

print("Quick sanity check (1 run)...")
res0 = run_single_experiment(random_state=0, verbose=True)

In [ ]:
# === 5. 10 runs: statistics summary ===

n_runs = 10
all_runs = []

for seed in range(n_runs):
    print(f"\n=== Run {seed+1}/{n_runs} (seed={seed}) ===")
    r = run_single_experiment(random_state=seed, verbose=False)
    print(f"QML train/test: {r['acc_tr_q']:.4f} / {r['acc_ts_q']:.4f}")
    print(f"LR  train/test: {r['acc_tr_lr']:.4f} / {r['acc_ts_lr']:.4f}")
    print(f"MLP train/test: {r['acc_tr_mlp']:.4f} / {r['acc_ts_mlp']:.4f}")
    all_runs.append(r)

records = []
for r in all_runs:
    records.append({
        "seed": r["seed"],
        "train_qml": r["acc_tr_q"],
        "test_qml":  r["acc_ts_q"],
        "train_lr":  r["acc_tr_lr"],
        "test_lr":   r["acc_ts_lr"],
        "train_mlp": r["acc_tr_mlp"],
        "test_mlp":  r["acc_ts_mlp"],
    })

runs_df = pd.DataFrame(records)
display(runs_df.head())

def mean_std(col):
    return runs_df[col].mean()*100, runs_df[col].std()*100

summary = {
    "Model": ["QML (1 qubit, ES λ=24, 60 gens)", "Logistic Regression", "MLP (50, 50)"],
    "Train Acc (mean ± std) [%]": [
        f"{mean_std('train_qml')[0]:.2f} ± {mean_std('train_qml')[1]:.2f}",
        f"{mean_std('train_lr')[0]:.2f}  ± {mean_std('train_lr')[1]:.2f}",
        f"{mean_std('train_mlp')[0]:.2f} ± {mean_std('train_mlp')[1]:.2f}",
    ],
    "Test Acc (mean ± std) [%]": [
        f"{mean_std('test_qml')[0]:.2f} ± {mean_std('test_qml')[1]:.2f}",
        f"{mean_std('test_lr')[0]:.2f}  ± {mean_std('test_lr')[1]:.2f}",
        f"{mean_std('test_mlp')[0]:.2f} ± {mean_std('test_mlp')[1]:.2f}",
    ],
    "Best train/test [%]": [
        f"{runs_df['train_qml'].max()*100:.2f} / {runs_df['test_qml'].max()*100:.2f}",
        f"{runs_df['train_lr'].max()*100:.2f}  / {runs_df['test_lr'].max()*100:.2f}",
        f"{runs_df['train_mlp'].max()*100:.2f} / {runs_df['test_mlp'].max()*100:.2f}",
    ],
    "Parameters (approx)": [3, 3, 2751],
}

summary_df = pd.DataFrame(summary)
print("\nSummary over 10 runs:")
display(summary_df)

In [ ]:
# === 6. Decision boundaries for best QML run ===

best_idx = runs_df['test_qml'].idxmax()
best_run = all_runs[best_idx]
print("Best QML run seed:", best_run["seed"])

X_tr = best_run["X_tr"]; y_tr = best_run["y_tr"]
X_ts = best_run["X_ts"]; y_ts = best_run["y_ts"]
qml_best = best_run["qml"]
lr_best = best_run["lr"]
mlp_best = best_run["mlp"]

# Rango similar al del paper
x_min, x_max = 0.0, 3.0
y_min, y_max = 0.0, 3.0

xx, yy = np.meshgrid(
    np.linspace(x_min, x_max, 200),
    np.linspace(y_min, y_max, 200)
)
grid = np.c_[xx.ravel(), yy.ravel()]

Z_qml = qml_best.predict(grid).reshape(xx.shape)
Z_lr  = lr_best.predict(grid).reshape(xx.shape)
Z_mlp = mlp_best.predict(grid).reshape(xx.shape)

def plot_decision(ax, Z, title):
    ax.contourf(xx, yy, Z, alpha=0.3)
    ax.scatter(X_tr[y_tr==0,0], X_tr[y_tr==0,1], c='red', marker='^', label='Class 0 (train)', alpha=0.6)
    ax.scatter(X_tr[y_tr==1,0], X_tr[y_tr==1,1], c='blue', marker='o', label='Class 1 (train)', alpha=0.6)
    ax.set_xlabel(r"$\sqrt{\mathrm{variance} + \mathrm{skewness}}$")
    ax.set_ylabel("Curtosis")
    ax.set_title(title)

fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharex=True, sharey=True)
plot_decision(axes[0], Z_qml, "Qubit QML (ES λ=24, 60 gens)")
plot_decision(axes[1], Z_mlp, "Multilayer Perceptron")
plot_decision(axes[2], Z_lr,  "Logistic Regression")
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper right")
plt.tight_layout()
plt.show()

In [ ]:
# === 7. Confusion matrix for best QML run ===
from itertools import product

y_pred_qml = qml_best.predict(X_ts)
cm = confusion_matrix(y_ts, y_pred_qml)

print("Confusion matrix (test set, best QML run):")
print(cm)

fig, ax = plt.subplots(figsize=(3,3))
im = ax.imshow(cm, cmap="Blues")
for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
    ax.text(j, i, cm[i, j], ha="center", va="center", color="black")
ax.set_xticks([0,1]); ax.set_yticks([0,1])
ax.set_xticklabels(["Pred 0", "Pred 1"])
ax.set_yticklabels(["True 0", "True 1"])
plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()